# Reading DB

In [1]:
import pandas as pd
from equity_db.api.mongo_connection import MongoAPI
from equity_db.query.asset_query import AssetQuery
from equity_db.query.db_query import ReadDB

In [2]:
# making our api connection
api = MongoAPI('test', 'crsp')

# making our class to construct our queries
reader = ReadDB(api)

Queries return a AssetQuery class which can be turned into a dataframe. It is also possible to caches the query's results by calling the set_save() method. We can also specify a trading calendar to join the query results onto. We can do this using set_calendar("cal_name").

In [3]:
start = pd.Timestamp(year=2019, month=1, day=1)
end = pd.Timestamp(year=2020, month=1, day=1)

# Here's an example query. We get back an AssetQuery object which can be turned into a DataFrame by using AssetQuery.df
combo_data: AssetQuery = reader.get_asset_data(
    assets=['AAPL', 'JNJ', 'MMM', 'MSFT', 'AMD', 'NVDA', 'KL', 'L' ,'HD'], # tickers to query
    # close, high, low, open, ticker, company name, and website
    fields=['prccd', 'prchd', 'prcld', 'prcod', 'tic', 'conm', 'weburl'],
    start = start, # start date
    end = end, # end date
    search_by='tic' # We are searching by ticker
)

In [4]:
# now lets turn the query into a DataFrame
df_of_combo_data: pd.DataFrame = combo_data.df
df_of_combo_data.sort_index()

conm             weburl     prchd   prcld  \
date       tic                                                                 
2019-01-02 AAPL               APPLE INC      www.apple.com  158.8500  154.23   
           AMD   ADVANCED MICRO DEVICES        www.amd.com   19.0000   17.98   
           HD            HOME DEPOT INC  www.homedepot.com  173.1800  169.05   
           JNJ        JOHNSON & JOHNSON        www.jnj.com  128.3800  126.28   
           KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   26.4050   25.80   
...                                 ...                ...       ...     ...   
2019-12-31 KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   44.4801   43.87   
           L                 LOEWS CORP      www.loews.com   52.5100   51.97   
           MMM                    3M CO         www.3m.com  176.5900  175.07   
           MSFT          MICROSOFT CORP  www.microsoft.com  157.7700  156.45   
           NVDA             NVIDIA CORP     www.nvidia.com  235.6750  230.13   

                  prcod   prccd  
date       tic                   
2019-01-02 AAPL  154.89  157.92  
           AMD    18.01   18.83  
           HD    169.71  172.41  
           JNJ   128.13  127.75  
           KL     26.17   26.09  
...                 ...     ...  
2019-12-31 KL     44.29   44.07  
           L      52.04   52.49  
           MMM   175.23  176.42  
           MSFT  156.77  157.70  
           NVDA  230.90  235.30  

[2268 rows x 6 columns]

In [5]:
# all static data is turned into a categorical type
df_of_combo_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2268 entries, (Timestamp('2019-07-05 00:00:00'), 'KL') to (Timestamp('2019-09-12 00:00:00'), 'NVDA')
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   conm    2268 non-null   category
 1   weburl  2268 non-null   category
 2   prchd   2268 non-null   float64 
 3   prcld   2268 non-null   float64 
 4   prcod   2268 non-null   float64 
 5   prccd   2268 non-null   float64 
dtypes: category(2), float64(4)
memory usage: 93.2 KB


In [6]:
# However we cant call .df twice because mongo queries return a generator like object which be read into memory once.
try:
    display(combo_data.df)
except ValueError as e:
    print('We hit ValueError')


We hit ValueError


Now let's do the same query but let's cache the query along with specifying the calendar.

In [7]:

# Same query as above
combo_data: AssetQuery = reader.get_asset_data(
    assets=['AAPL', 'JNJ', 'MMM', 'MSFT', 'AMD', 'NVDA', 'KL', 'L' ,'HD'], # tickers to query
    # close, high, low, open, ticker, company name, and website
    fields=['prccd', 'prchd', 'prcld', 'prcod', 'tic', 'conm', 'weburl'],
    start = start, # start date
    end = end, # end date
    search_by='tic' # in this query we are searching by ticker
)


In [8]:
# Now lets call set save telling the AssetQuery to save the query
# We have also filtered the dates in the dataframe to only dates the NYSE was trading
# we can use any calendar in pandas_market_calenders or along with "365" which includes every single possible day
df_of_combo_data: pd.DataFrame = combo_data.set_save().set_calendar('NYSE').df
df_of_combo_data.sort_index()

conm             weburl     prchd   prcld  \
date       tic                                                                 
2019-01-02 AAPL               APPLE INC      www.apple.com  158.8500  154.23   
           AMD   ADVANCED MICRO DEVICES        www.amd.com   19.0000   17.98   
           HD            HOME DEPOT INC  www.homedepot.com  173.1800  169.05   
           JNJ        JOHNSON & JOHNSON        www.jnj.com  128.3800  126.28   
           KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   26.4050   25.80   
...                                 ...                ...       ...     ...   
2019-12-31 KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   44.4801   43.87   
           L                 LOEWS CORP      www.loews.com   52.5100   51.97   
           MMM                    3M CO         www.3m.com  176.5900  175.07   
           MSFT          MICROSOFT CORP  www.microsoft.com  157.7700  156.45   
           NVDA             NVIDIA CORP     www.nvidia.com  235.6750  230.13   

                  prcod   prccd  
date       tic                   
2019-01-02 AAPL  154.89  157.92  
           AMD    18.01   18.83  
           HD    169.71  172.41  
           JNJ   128.13  127.75  
           KL     26.17   26.09  
...                 ...     ...  
2019-12-31 KL     44.29   44.07  
           L      52.04   52.49  
           MMM   175.23  176.42  
           MSFT  156.77  157.70  
           NVDA  230.90  235.30  

[2268 rows x 6 columns]

In [9]:
# now we can use .df an unlimited amount of times with no problems
combo_data.df
combo_data.df
combo_data.df
combo_data.df
combo_data.df.sort_index()


conm             weburl     prchd   prcld  \
date       tic                                                                 
2019-01-02 AAPL               APPLE INC      www.apple.com  158.8500  154.23   
           AMD   ADVANCED MICRO DEVICES        www.amd.com   19.0000   17.98   
           HD            HOME DEPOT INC  www.homedepot.com  173.1800  169.05   
           JNJ        JOHNSON & JOHNSON        www.jnj.com  128.3800  126.28   
           KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   26.4050   25.80   
...                                 ...                ...       ...     ...   
2019-12-31 KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   44.4801   43.87   
           L                 LOEWS CORP      www.loews.com   52.5100   51.97   
           MMM                    3M CO         www.3m.com  176.5900  175.07   
           MSFT          MICROSOFT CORP  www.microsoft.com  157.7700  156.45   
           NVDA             NVIDIA CORP     www.nvidia.com  235.6750  230.13   

                  prcod   prccd  
date       tic                   
2019-01-02 AAPL  154.89  157.92  
           AMD    18.01   18.83  
           HD    169.71  172.41  
           JNJ   128.13  127.75  
           KL     26.17   26.09  
...                 ...     ...  
2019-12-31 KL     44.29   44.07  
           L      52.04   52.49  
           MMM   175.23  176.42  
           MSFT  156.77  157.70  
           NVDA  230.90  235.30  

[2268 rows x 6 columns]

### Can also pull data for static data only query's

In [10]:
static_data: AssetQuery = reader.get_asset_data(
    assets=['AAPL', 'JNJ', 'MMM', 'MSFT', 'AMD', 'NVDA', 'KL', 'L' ,'HD'],
    fields=['tic', 'conm', 'weburl', 'cusip', 'lpermno'], # static fields
    search_by='tic' # in this query we are using ticker as our asset id
)

In [11]:
# now lets turn the searched data into a DataFrame
df_of_static_data = static_data.df
df_of_static_data.sort_index()

,cusip,conm,weburl,lpermno
tic,,,,
AAPL,037833100,APPLE INC,www.apple.com,14593
AMD,007903107,ADVANCED MICRO DEVICES,www.amd.com,61241
HD,437076102,HOME DEPOT INC,www.homedepot.com,66181
JNJ,478160104,JOHNSON & JOHNSON,www.jnj.com,22111
KL,49741E100,KIRKLAND LAKE GOLD LTD,www.klgold.com,16865
L,540424108,LOEWS CORP,www.loews.com,26710
MMM,88579Y101,3M CO,www.3m.com,22592
MSFT,594918104,MICROSOFT CORP,www.microsoft.com,10107
NVDA,67066G104,NVIDIA CORP,www.nvidia.com,86580


In [12]:
# we are searching by permno here (called lpermno in the crsp compustat linked dataset)
static_data_lpermno: AssetQuery = reader.get_asset_data(
     assets=['14593', '61241', '66181', '22111', '16865', '26710', '22592', '10107', '86580'],
     fields=['tic', 'conm', 'weburl'], # static fields
     search_by='lpermno' # in this query we are using ticker as our asset id
     )

In [13]:
# now lets turn the searched data into a DataFrame
df_of_static_data_lpermno = static_data_lpermno.df
df_of_static_data_lpermno.sort_index()

,conm,tic,weburl
lpermno,,,
10107,MICROSOFT CORP,MSFT,www.microsoft.com
14593,APPLE INC,AAPL,www.apple.com
16865,KIRKLAND LAKE GOLD LTD,KL,www.klgold.com
22111,JOHNSON & JOHNSON,JNJ,www.jnj.com
22592,3M CO,MMM,www.3m.com
26710,LOEWS CORP,L,www.loews.com
61241,ADVANCED MICRO DEVICES,AMD,www.amd.com
66181,HOME DEPOT INC,HD,www.homedepot.com
86580,NVIDIA CORP,NVDA,www.nvidia.com
